In [1]:
!pip install minio delta-spark==2.2.0

In [2]:
from minio import Minio
import io
import logging
import sys
import traceback
from datetime import datetime

import urllib3

from minio import Minio
from pyspark import SparkContext
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import unix_timestamp
import minio
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [3]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.driver.host', 'd63a786ad49c'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.2-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.2-bi

In [4]:
df_orders = spark.read.csv("s3a://donnes-capteurs/donnes_capteurs_13-05-23.csv", header=True)


In [5]:
df_orders.limit(20).toPandas()

,timestamp,entrance_amount,exit_amount,temperature,humidity,parking_entrance,parking_exit,parking_actual_vehicle
0,2023-05-13 11:12:49.597652,13,41,20.978106526032036,47.53296947848155,2,1,390
1,2023-05-13 11:12:49.597652,33,9,19.75909362330829,37.612915439352506,2,2,264
2,2023-05-13 11:12:49.597652,41,35,24.084290631587795,52.65643133784484,1,2,192
3,2023-05-13 11:12:49.597652,9,27,24.02563620852396,48.24337238214185,2,2,96
4,2023-05-13 11:12:49.597652,28,2,23.304928413054203,66.58388432907365,3,2,473
5,2023-05-13 11:12:49.597652,6,33,16.459926354265733,62.705480007420746,2,1,165
6,2023-05-13 11:12:49.597652,44,17,27.357699370753625,61.29498554714034,1,4,214
7,2023-05-13 11:12:49.597652,38,36,21.147753720623495,48.17493726001879,2,2,348
8,2023-05-13 11:12:49.597652,31,6,12.387284594709788,47.2316581689888,2,1,396
9,2023-05-13 11:12:49.597652,36,32,24.504403457766717,55.177506025883275,3,2,293


In [6]:
spark.range(3000).write.format("delta").save("s3a://donnes-capteurs/testfile.txt", mode="overwrite")

In [8]:
sc = spark.sparkContext
fdd = sc.textFile("s3a://donnes-capteurs/testfile.txt")


In [9]:
def create_model(iter, features_cols, labelCol):
    # create LinearRegression estimator and set parameters
    lr = LinearRegression(maxIter=iter,
                          featuresCol=features_cols,
                          labelCol=labelCol)
    return lr


def set_train_and_validation_ds(data, seed):
    return data.randomSplit([0.7, 0.3], seed=seed)


def test_model(spark, model):
    # Methode pour tester le modèle avec un jeu de données test
    testData = spark.read.format("libsvm").load("s3a://donnes-capteurs/testfile.txt")
    predictions = model.transform(testData)
    predictions.show()


def save_model(model):
    # Methode pour sauvegarder le modèle vers un lieu spécifié
    model.save("saved_model/model_test")

In [11]:
columns_necessaire = ["entrance_amount", "exit_amount","parking_entrance", "parking_exit"]
target = ["parking_actual_vehicle"]


TypeError: Invalid param value given for param "featuresCol". Could not convert <class 'list'> to string type

In [12]:
df_orders_2 = df_orders[columns_necessaire]
df_target = df_orders[target]
create_model(10, df_orders_2, df_target)

TypeError: Invalid param value given for param "featuresCol". Could not convert <class 'pyspark.sql.dataframe.DataFrame'> to string type